## Resume Matching Scanner

##### The goal of this project is to help job seekers to verify if their resume is matching with the job description they are interested

In [1]:
import PyPDF2
import textract
import re
import string
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import spacy
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.feature_extraction.text import TfidfVectorizer


from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from wordcloud import WordCloud
from nltk.probability import FreqDist
import operator

2023-06-04 17:41:58.154983: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-04 17:41:58.280357: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-04 17:41:58.280384: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-04 17:41:58.868646: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
set(stopwords.words('english'))

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

### Getting information from the CV

In [5]:
def get_info_cv(cv_path):
    pdf_file = open(cv_path,'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    num_pages = len(pdf_reader.pages)
    print('The number of pages in the CV are: ',num_pages)
    
    count = 0
    text = ""

    while count < num_pages:
        pageObj = pdf_reader.pages[count]
        count +=1
        text += pageObj.extract_text()
    
    return text

In [9]:
cv=get_info_cv('apple-nlp-engineer-resume-example.pdf')

The number of pages in the CV are:  1


In [10]:
print(cv)

Twila Goldston
NLP Engineer
As an experienced Natural Language Processing (NLP) Engineer with over 3 years of hands-on experience, I have 
developed strong expertise in the development and deployment of NLP solutions for a variety of industries. My key skills 
include designing, developing, and deploying NLP models and pipelines, as well as automating complex NLP tasks.
I have successfully developed and implemented NLP solutions for a variety of tasks such as entity recognition and 
extraction, sentiment analysis, information retrieval, text summarization, and machine translation. I have also worked 
on developing custom NLP pipelines, utilizing deep learning models such as LSTM, Transformer, and BERT. Additionally, 
I have strong knowledge of various NLP libraries such as Spacy, NL TK, Gensim, and Scikit-learn, as well as programming 
languages such as Python and Java.
Throughout my career, I have been involved in a number of successful projects, such as developing a text summarizatio

### Getting information from the Job Offer

In [11]:
f = open("nlp_engineer_job_description.txt", "r")
offer=f.read()

In [12]:
print(offer)

Job brief

We are looking for a Natural Language Processing Engineer to help us improve our NLP products and create new NLP applications.

NLP Engineer responsibilities include transforming natural language data into useful features using NLP techniques to feed classification algorithms. To succeed in this role, you should possess outstanding skills in statistical analysis, machine learning methods and text representation techniques.

Your ultimate goal is to develop efficient self-learning NLP applications.
Responsibilities

    Study and transform data science prototypes
    Design NLP applications
    Select appropriate annotated datasets for Supervised Learning methods
    Use effective text representations to transform natural language into useful features
    Find and implement the right algorithms and tools for NLP tasks
    Develop NLP systems according to requirements
    Train the developed model and run evaluation experiments
    Perform statistical analysis of results and r

### Cleaning each document

In [13]:
def clean_text(text):
    text = text.replace('\n', ' ')
    text = text.replace(',', ' ')
    text = text.replace('...', ' ')
    text = text.replace('|', ' ')
    return text

In [14]:
cv_cleaned=clean_text(cv)
offer_cleaned=clean_text(offer)

In [15]:
cv_cleaned

'Twila Goldston NLP Engineer As an experienced Natural Language Processing (NLP) Engineer with over 3 years of hands-on experience  I have  developed strong expertise in the development and deployment of NLP solutions for a variety of industries. My key skills  include designing  developing  and deploying NLP models and pipelines  as well as automating complex NLP tasks. I have successfully developed and implemented NLP solutions for a variety of tasks such as entity recognition and  extraction  sentiment analysis  information retrieval  text summarization  and machine translation. I have also worked  on developing custom NLP pipelines  utilizing deep learning models such as LSTM  Transformer  and BERT. Additionally   I have strong knowledge of various NLP libraries such as Spacy  NL TK  Gensim  and Scikit-learn  as well as programming  languages such as Python and Java. Throughout my career  I have been involved in a number of successful projects  such as developing a text summarizati

In [16]:
offer_cleaned

'Job brief  We are looking for a Natural Language Processing Engineer to help us improve our NLP products and create new NLP applications.  NLP Engineer responsibilities include transforming natural language data into useful features using NLP techniques to feed classification algorithms. To succeed in this role  you should possess outstanding skills in statistical analysis  machine learning methods and text representation techniques.  Your ultimate goal is to develop efficient self-learning NLP applications. Responsibilities      Study and transform data science prototypes     Design NLP applications     Select appropriate annotated datasets for Supervised Learning methods     Use effective text representations to transform natural language into useful features     Find and implement the right algorithms and tools for NLP tasks     Develop NLP systems according to requirements     Train the developed model and run evaluation experiments     Perform statistical analysis of results and 

### Vectorizing each document

##### Here we are creating our TF-IDF vectorizer and then obtaining a vocabulary based in the job description

In [17]:
vectorizer = TfidfVectorizer()
vectorizer.fit([offer_cleaned])

TfidfVectorizer()

#### Vectorization of the job offer

In [23]:
X = vectorizer.transform([offer_cleaned])

In [24]:
Y=vectorizer.transform([cv])

### Cosine Similarity between each document

In [25]:
cos=cosine_similarity(X,Y)

In [31]:
print('The similarity between the job description and the cv is: ', round(cos[0][0]*100,2),'%')

The similarity between the job description and the cv is:  70.66 %
